In [2]:
from collections import defaultdict

year = 2024; day = 4

In [3]:
from aocd import get_data, submit
from helpers.helpers import parse_single_matrix

data = get_data(year=year, day=day)

data = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""
data = data.strip()

m = parse_single_matrix(data)


In [4]:
# original solution

def find_XMAS(y, x, m):
    directions = [
        (0, 1),
        (1, 0),
        (1, 1),
        (-1, 1),
        (-1, 0),
        (0, -1),
        (1, -1),
        (-1, -1),
    ]
    count = 0
    for dx, dy in directions:
        found = True
        for i in range(1, len("XMAS")):
            if (y + i*dy) < 0 or (x + i*dx) < 0:
                found = False
                break
            if (y + i*dy) >= m.shape[0] or (x + i*dx) >= m.shape[1]:
                found = False
                break
            if m[y + i*dy, x + i*dx] != "XMAS"[i]:
                found = False
                break
        if found:
            count += 1
    return count

In [103]:
direction_mapping = {
    (1,0): "DOWN",
    (0,1): "RIGHT",
    (1,1): "DOWNRIGHT",
    (-1,1): "UPRIGHT",
    (-1,0): "UP",
    (0,-1): "LEFT",
    (1,-1): "DOWNLEFT",
    (-1,-1): "UPLEFT"
}

In [114]:
from collections import defaultdict
#refactored for reusing later
match_count = defaultdict(int)
from itertools import product
def check_XMAS(y, x, m):
    count = 0
    N = len("XMAS")
    for dy, dx in product([-1, 0, 1], [-1, 0, 1]):
        if dy == dx == 0: continue
        if not ((0 <= y+dy*(N-1) < m.shape[0]) and (0 <= x+dx*(N-1) < m.shape[1])):
            continue
        if all(m[y+i*dy, x+i*dx] == "XMAS"[i] for i in range(N)):
            match_count[direction_mapping[(dy, dx)]] += 1
            print(y, x, direction_mapping[(dy, dx)])
            count += 1
    return count

In [115]:
count = 0
for y in range(m.shape[0]):
    for x in range(m.shape[1]):
        if m[y, x] == "X":
            count += check_XMAS(y, x, m)
count

0 4 DOWNRIGHT
0 5 RIGHT
1 4 LEFT
3 9 DOWNLEFT
3 9 DOWN
4 0 RIGHT
4 6 UP
4 6 LEFT
5 0 UPRIGHT
5 6 UPLEFT
9 1 UPRIGHT
9 3 UPLEFT
9 3 UPRIGHT
9 5 UPLEFT
9 5 UPRIGHT
9 5 RIGHT
9 9 UPLEFT
9 9 UP


18

In [7]:
count = 0
for y in range(m.shape[0]):
    for x in range(m.shape[1]):
        if m[y, x] == "X":
            count += check_XMAS(y, x, m)
count

18

In [8]:
submit(count, part="a", year=year, day=day)

Part a already solved with different answer: 2571


In [9]:
def find_XMAS(y, x, m):
    count = 0

    if y - 1 < 0 or x - 1 < 0:
        return count
    if y + 1 >= m.shape[0] or x + 1 >= m.shape[1]:
        return count

    left = False
    right = False
    if m[y-1, x-1] == "M" and m[y+1, x+1] == "S":
        left = True
    if m[y-1, x-1] == "S" and m[y+1, x+1] == "M":
        left = True
    if m[y-1, x+1] == "M" and m[y+1, x-1] == "S":
        right = True
    if m[y-1, x+1] == "S" and m[y+1, x-1] == "M":
        right = True

    if left and right:
        count = 1
        # print(f"found {count} at {y, x}")
    return count


In [10]:
count = 0
for y in range(m.shape[0]):
    for x in range(m.shape[1]):
        if m[y, x] == "A":
            count += find_XMAS(y, x, m)
count

9

In [11]:
submit(count, part="b", year=year, day=day)

Part b already solved with different answer: 1992


In [75]:
X = 1; M = 10; A = 100; S = 1000
import numpy as np
chars_to_ints = {
    "X": X,
    "M": M,
    "A": A,
    "S": S,
}

# converted = chars_to_ints[c] if c in chars_to_ints else c for c in data

m = parse_single_matrix(data)
converted = np.vectorize(lambda x: chars_to_ints[x] if x in chars_to_ints else x)(m)

search_patterns = [
    np.array([[X, M, A, S]]),
    np.array([[X],[M],[A],[S]]),
    np.array([[S, A, M, X]]),
    np.array([[S],[A],[M],[X]]),
    np.array([
        [X, 0, 0, 0],
        [0, M, 0, 0],
        [0, 0, A, 0],
        [0, 0, 0, S],
    ]),
    np.array([
        [S, 0, 0, 0],
        [0, A, 0, 0],
        [0, 0, M, 0],
        [0, 0, 0, X],
    ]),
    np.array([
        [0, 0, 0, X],
        [0, 0, M, 0],
        [0, A, 0, 0],
        [S, 0, 0, 0]
    ]),
        np.array([
        [0, 0, 0, S],
        [0, 0, A, 0],
        [0, M, 0, 0],
        [X, 0, 0, 0]
    ])
]

In [76]:
search_patterns[0].dtype


dtype('int64')

In [77]:
%pip install scipy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [78]:
from scipy.signal import convolve2d, correlate2d


In [79]:
search_value = sum(i*i for i in chars_to_ints.values())
search_value

1010101

In [121]:
count = 0
all_matches = np.zeros_like(converted)
# for pattern in search_patterns:
pattern = search_patterns[4]
matches = (correlate2d(converted, pattern, "same") == search_value).astype("int")
# all_matches += matches
print(matches, sum(sum(matches)), match_count["DOWN_RIGHT"])

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]] 1 0


In [118]:
pattern

array([[   1],
       [  10],
       [ 100],
       [1000]])

In [66]:
# 0 4 DOWNRIGHT
# 0 5 RIGHT
# 1 4 LEFT
# 3 9 DOWNLEFT
# 3 9 DOWN
# 4 0 RIGHT
# 4 6 UP
# 4 6 LEFT
# 5 0 UPRIGHT
# 5 6 UPLEFT
# 9 1 UPRIGHT
# 9 3 UPLEFT
# 9 3 UPRIGHT
# 9 5 UPLEFT
# 9 5 UPRIGHT
# 9 5 RIGHT
# 9 9 UPLEFT
# 9 9 UP

204

In [100]:
# print(data[0][6])


data[0]
print(data)

MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX


In [52]:
np.diag(np.ones(5))

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [56]:
convolve2d(a, np.diag(np.ones(5)), mode="full")

array([[1., 2., 0., 0., 0., 0.],
       [3., 5., 2., 0., 0., 0.],
       [0., 3., 5., 2., 0., 0.],
       [0., 0., 3., 5., 2., 0.],
       [0., 0., 0., 3., 5., 2.],
       [0., 0., 0., 0., 3., 4.]])